In [1]:
import os
import time
import math
import helper
import datetime
import requests
import bs4 as bs
import numpy as np
import pandas as pd
import pickle as pkl
import datetime as dt
import tensorflow as tf
from sklearn import metrics
from matplotlib import style
import yahoo_finance as yahoo
from datetime import timedelta
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
import matplotlib.dates as mdates
from matplotlib.finance import candlestick_ohlc
from IPython.display import display, Math, Latex

style.use('ggplot')

C:\Users\Khanh\Anaconda3\envs\py35\lib\site-packages\matplotlib\cbook.py:136: MatplotlibDeprecationWarning: The finance module has been deprecated in mpl 2.0 and will be removed in mpl 2.2. Please use the module mpl_finance instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [2]:
df = pd.read_csv('./data/ibm/full_ibm.csv')
df['Date.1'] = pd.to_datetime(df['Date.1'])
df.set_index('Date.1', inplace=True)
df.drop(['Date','High', 'Adj Close', 'NDX', 'SP500', 'Volume', 'Low'], axis=1, inplace=True)
df.head()

,Open,Close,DJIA
Date.1,,,
2000-01-03,112.4375,116.0000,11357.509766
2000-01-04,114.0000,112.0625,10997.929688
2000-01-05,112.9375,116.0000,11122.650391
2000-01-06,118.0000,114.0000,11253.259766
2000-01-07,117.2500,113.5000,11522.559570


In [3]:
df.index

DatetimeIndex(['2000-01-03', '2000-01-04', '2000-01-05', '2000-01-06',
               '2000-01-07', '2000-01-10', '2000-01-11', '2000-01-12',
               '2000-01-13', '2000-01-14',
               ...
               '2016-12-16', '2016-12-19', '2016-12-20', '2016-12-21',
               '2016-12-22', '2016-12-23', '2016-12-27', '2016-12-28',
               '2016-12-29', '2016-12-30'],
              dtype='datetime64[ns]', name='Date.1', length=4277, freq=None)

In [4]:
df['ma7'] = df['Close'].rolling(window=7).mean()
df['ma30'] = df['Close'].rolling(window=30).mean()
df['std30'] = df['Close'].rolling(window=30).std()
df.tail()

,Open,Close,DJIA,ma7,ma30,std30
Date.1,,,,,,
2016-12-23,167.000000,166.710007,19933.810547,167.161429,163.586334,3.291480
2016-12-27,166.979996,167.139999,19945.039062,167.035714,163.782000,3.323355
2016-12-28,167.289993,166.190002,19833.679688,166.958572,164.048000,3.178183
2016-12-29,166.020004,166.600006,19819.779297,166.947146,164.312334,3.042335
2016-12-30,166.440002,165.990005,19762.599609,166.717146,164.535667,2.903699


In [5]:
df_train = df.drop(['ma7', 'Close', 'ma30'],axis=1)
df_train['ma7_1'] = df['ma7'].shift(periods = 11, axis=0)
df_train['ma7_2'] = df['ma7'].shift(periods = 18, axis=0)
df_train['ma7_3'] = df['ma7'].shift(periods = 25, axis=0)
df_train['ma7_4'] = df['ma7'].shift(periods = 32, axis=0)
df_train['ma30_1'] = df['ma7'].shift(periods = 62, axis=0)
df_train['ma30_2'] = df['ma7'].shift(periods = 92, axis=0)
df_train['ma30_3'] = df['ma7'].shift(periods = 112, axis=0)
df_train['ma30_4'] = df['ma7'].shift(periods = 142, axis=0)
df_train['Close_1'] = df['Close'].shift(periods = 1, axis=0)
df_train['Close_2'] = df['Close'].shift(periods = 2, axis=0)
df_train['Close_3'] = df['Close'].shift(periods = 3, axis=0)
print (df_train.shape)
df_train.tail()

(4277, 14)


,Open,DJIA,std30,ma7_1,ma7_2,ma7_3,ma7_4,ma30_1,ma30_2,ma30_3,ma30_4,Close_1,Close_2,Close_3
Date.1,,,,,,,,,,,,,,
2016-12-23,167.000000,19933.810547,3.291480,161.771430,162.714286,158.895715,153.445711,155.241427,162.392857,157.824286,152.801428,167.059998,167.330002,167.600006
2016-12-27,166.979996,19945.039062,3.323355,162.385716,162.975714,159.692858,153.605711,155.729998,161.992857,158.555714,152.949999,166.710007,167.059998,167.330002
2016-12-28,167.289993,19833.679688,3.178183,163.197143,162.554286,160.057144,154.667140,156.252856,161.764287,159.417143,153.077142,167.139999,166.710007,167.059998
2016-12-29,166.020004,19819.779297,3.042335,164.378570,162.175716,160.257143,155.998570,156.727142,161.705715,159.904286,153.242857,166.190002,167.139999,166.710007
2016-12-30,166.440002,19762.599609,2.903699,165.617142,161.870001,160.795713,156.832857,156.941428,161.414285,160.482858,153.197141,166.600006,166.190002,167.139999


### Network

In [6]:
## params
N_FEATURES = 126 # number of input features
X = tf.placeholder(tf.float32, [None, N_FEATURES])
Y = tf.placeholder(tf.float32, [None, 1])

In [7]:
def new_fc_layer(in_layer, n_input_features, n_output_layer, activation=''):
    weights = tf.Variable(tf.truncated_normal([n_input_features, n_output_layer], stddev=0.1))
    biases = tf.Variable(tf.constant(value=1, dtype=tf.float32, shape = [n_output_layer]))
    out_layer = tf.matmul(in_layer, weights) + biases
    activation = activation.lower()
    if(activation == 'relu'):
        out_layer = tf.nn.relu(out_layer)
    elif(activation == 'tanh'):
        out_layer = tf.nn.tanh(out_layer)
    elif(activation == 'sigmoid'):
        out_layer = tf.nn.sigmoid(out_layer)
    return out_layer, weights, biases
## Hidden layer 1
N_HIDDEN_L1 = 1
hidden_layer_1, weight_1, biases_1 = new_fc_layer(X, N_FEATURES, N_HIDDEN_L1, 'tanh')

print (hidden_layer_1)


Tensor("Tanh:0", shape=(?, 1), dtype=float32)


In [8]:
learning_rate = tf.placeholder(tf.float32, shape=[])
def decay_learning_rate(learning_step, total_step):
    ini = 1e-3
    if(learning_step <= 3000):
        return ini
    return ini*(1-(learning_step-3000)/total_step)

In [9]:
def plot_stock(index, x1, x2, dir_path, rmse, mae, epoch, plot, train):
    if(train):
        folder = dir_path + 'train/'
    else:
        folder = dir_path + 'test/'
    if not os.path.exists(folder):
        os.makedirs(folder)
    years = mdates.YearLocator()   # every year
    months = mdates.MonthLocator()  # every month
    yearsFmt = mdates.DateFormatter('%Y')
    plt.close()
    plt.clf()
    fig, ax = plt.subplots(figsize=(10,5))
    if(plot=='abs'):
        abs_error = np.abs(x1 - x2)
        ax.plot(index, abs_error, color = 'b', label='Absolute error')
        ax.text(index[1], abs_error.max() - 0.01, "MAE={:7.5f}".format(mae), style='italic',
            bbox={'facecolor':'blue', 'alpha':0.5, 'pad':10})
    elif(plot=='cost'):
        ax.plot(index, x1, color = 'b', label='Cost value')
    elif(plot=='rmse'):
        ax.plot(index, x1, color = 'b', label='RMSE value')
    elif(plot=='prediction'):  
        ax.plot(index, x1, color = 'b', label='Acutal Values')
        ax.plot(index, x2, color = 'r', label='Predicted Values')
        ax.text(index[1], x1.min() + 0.1, "RMSE={:7.5f}".format(rmse), style='italic',
            bbox={'facecolor':'blue', 'alpha':0.5, 'pad':10})
    # format the ticks
    ax.xaxis.set_major_locator(years)
    ax.xaxis.set_major_formatter(yearsFmt)
    ax.xaxis.set_minor_locator(months)
    datemin = datetime.date(index.date.min().year, 1, 1)
    datemax = datetime.date(index.date.max().year + 1, 1, 1)
    ax.set_xlim(datemin, datemax)
    ax.format_xdata = mdates.DateFormatter('%d-%m-%Y')
    ax.set_xlabel('Time')
    ax.set_ylabel('norm price')
    ax.legend(loc='upper right', shadow=False)
    ax.grid(True)
    fig.autofmt_xdate()
     
    if(plot == 'abs'):
        plt.savefig('{0}Abs_error_{1}th_epoch'.format(folder, epoch), dpi=1000)
    elif(plot =='cost'):
        plt.savefig('{0}cost_value'.format(folder), dpi=1000)
    elif(plot=='rmse'):
        plt.savefig('{0}rmse_value'.format(folder), dpi=1000)
    elif(plot=='prediction'):
        plt.savefig('{0}{1}th_epoch'.format(folder, epoch), dpi=1000)
    plt.clf()
    plt.close()  

In [10]:
def cal_metrics(true, pred):
    rmse = math.sqrt(metrics.mean_squared_error(pred, true))
    mae = metrics.mean_absolute_error(pred, true)
    hit = np.mean(helper.step((pred[1:] - true[:-1])*(true[1:] - true[:-1])))
    return rmse, mae, hit

In [11]:
BATCH_SIZE = 30
TRAINING_EPOCHS = 10000
SAMPLING_RATE = 500
## Helper function for optimization
def optimize(train_x, train_y, test_x, test_y, n_epochs, batch_size, session, saver, dir_path, n_ensemble):
    n_samples = train_x.shape[0]
    n_iterations = np.int(np.floor(n_samples/batch_size))+1
    start_time = time.time()
    cost_history = np.empty(shape=[1],dtype=float)
    train_metrics = np.zeros(shape=(np.int(n_epochs // SAMPLING_RATE)+1,3), dtype=float)
    test_metrics = np.zeros(shape=(np.int(n_epochs // SAMPLING_RATE)+1,3), dtype=float)
    train_prediction_record = np.zeros(shape=(np.int(n_epochs // SAMPLING_RATE)+1,train_y.shape[0], train_y.shape[1]))
    test_prediction_record = np.zeros(shape=(np.int(n_epochs // SAMPLING_RATE)+1,test_y.shape[0], test_y.shape[1]))
    train_y_matrix = train_y.as_matrix()
    best_iteration = 0
    previous_mse = 1000000
    print ("Training.......")
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    model_path = "{0}model_{1}/".format(dir_path, n_ensemble)
    if not os.path.exists(model_path):
        os.makedirs(model_path)
    for epoch in np.arange(n_epochs+1):
        for itr in np.arange(n_iterations):
            start = (itr * batch_size) % (n_samples )
            batch_x, batch_y = train_x[start:start + batch_size], train_y_matrix[start:start + batch_size]
            lr = decay_learning_rate(epoch, n_epochs)
            feed_dict_train = {X: batch_x, Y: batch_y, learning_rate: lr}
            _, c = session.run([optimizer, cost], feed_dict=feed_dict_train)
        prediction = make_prediction(test_x, test_y, session, BATCH_SIZE)
        r, _, _ = cal_metrics(test_y.as_matrix(), prediction)
        if(r < previous_mse):
            saver.save(session, "{}model.ckpt".format(model_path))
            previous_mse = r 
            best_iteration = epoch
        if(epoch >= SAMPLING_RATE):
            cost_history = np.append(cost_history,c)
        
            
        if(epoch % SAMPLING_RATE == 0):
            counter = np.int(epoch // SAMPLING_RATE)

    
            test_prediction_record[counter] = prediction
            
#             print ("-- Elapsed time -- Epoch -- Cost value -- ")
#             print ("-- {:12.6f} -- {:5d} -- {:10.5f} -- ".format((time.time() - start_time), 
#                                                                                     epoch, 
#                                                                                     c, 
#                                                                                     ))
#             print ("-- Making training prediction at {}th epoch".format(epoch))
            prediction = make_prediction(train_x, train_y, session, BATCH_SIZE)  
            
            train_prediction_record[counter] = prediction 
#             print ("-- Making testing prediction at {}th epoch".format(epoch))
            
        
#             Draw weights of convolutional layer
#             if(epoch % (n_epochs/2) == 0):
#                 plot_conv_weights(session, conv_weights[0], 'conv_1', 1, epoch)
#                 plot_conv_weights(session, conv_weights[1], 'conv_2', 1, epoch)
#                 plot_conv_weights(session, conv_weights[2], 'conv_3', 1, epoch)
                  
#         Save model in folder rnn_model
        
#         saver.save(sess, 'rnn_model/new_cnn')
        
#         print running time and output cost value graph
    running_time = time.time() - start_time
    print ("---Total Running time: %s seconds ---" % (running_time))
    print ('*'*50)
    return train_prediction_record, test_prediction_record, running_time, best_iteration 
    
    
## Helper function to print confusion matrix
def make_prediction(test_x, test_y, sess, batch_size):
    start_time = time.time()
    n_samples = test_x.shape[0]
    n_iterations = np.int(np.floor(n_samples/batch_size))+1 
    pred = np.zeros((n_samples, test_y.shape[1]))
#     true = test_y.as_matrix()
    for itr in np.arange(n_iterations):
        start = (itr * batch_size) % (n_samples)
        batch_x = test_x[start:start + batch_size]
        feed_dict_test = {X: batch_x}
        pred[start:start + batch_size] = sess.run(y_pred, feed_dict=feed_dict_test)
#     pred = (pred + 1) * (true.max() - true.min()) / 2 + true.min()
#     rmse = math.sqrt(metrics.mean_squared_error(pred, true))
#     mae = metrics.mean_absolute_error(pred, true)
#     hit = np.mean(helper.step((pred[1:] - true[:-1])*(true[1:] - true[:-1])))
#     print ("RMSE = {}".format(rmse))
#     print ("MAE = {}".format(mae))
#     print ("Hit rate = {}".format(hit))
#     print ('*'*50)
#     return rmse, mae, hit, pred 
    return pred



### Trignometric 

In [12]:
def train_ensemble(n_ensemble, train_x, train_y, test_x, test_y, n_epochs, batch_size, d):
    
    n = np.int(n_epochs // SAMPLING_RATE)+1
    train_pred = np.zeros(shape=(n,train_y.shape[0], train_y.shape[1]))
    test_pred = np.zeros(shape=(n,test_y.shape[0], test_y.shape[1]))
    train_metrics = np.zeros(shape=(n,3), dtype=float)
    test_metrics = np.zeros(shape=(n,3), dtype=float)
    running_times = np.zeros(shape=(n_ensemble,1), dtype=float)
    best_iteration = np.zeros(shape=(n_ensemble,1), dtype=int)
    best_pred = np.zeros(shape=test_y.shape)
    pred_time = 0
    for i in range(n_ensemble):
        print ('='*50)
        print ("Esemble {}".format(i))
        saver = tf.train.Saver()
        temp_train_pred = np.zeros(shape=(n,train_y.shape[0], train_y.shape[1]))
        temp_test_pred = np.zeros(shape=(n,test_y.shape[0], test_y.shape[1]))
#         temp_train_metrics = np.zeros(shape=(n,3), dtype=float)
#         temp_test_metrics = np.zeros(shape=(n,3), dtype=float)
        with tf.Session() as sess:
            
            sess.run(tf.global_variables_initializer())
            temp_train_pred,temp_test_pred, running_times[i,0], best_iteration[i,0] = optimize(x_train, y_train, x_test, y_test , n_epochs, batch_size, sess, saver, d, i)
#         train_metrics = train_metrics + (temp_train_metrics - train_metrics)/(i+1)
#         test_metrics = test_metrics + (temp_test_metrics - test_metrics)/(i+1)
#         temp_train_metrics[i, 0]
        train_pred = train_pred + (temp_train_pred - train_pred)/(i+1)
        test_pred = test_pred + (temp_test_pred - test_pred)/(i+1)
    for i in range(n):
        train_metrics[i,0], train_metrics[i,1], train_metrics[i,2] = cal_metrics(train_y.as_matrix(), train_pred[i])
        test_metrics[i,0], test_metrics[i,1], test_metrics[i,2] = cal_metrics(test_y.as_matrix(), test_pred[i])
    for i in np.arange(0, n, 2):
        plot_stock(train_y.index, train_y, train_pred[i], d, train_metrics[i,0], train_metrics[i,1], i*SAMPLING_RATE, 'prediction', True)
        plot_stock(train_y.index, train_y, train_pred[i], d, train_metrics[i,0], train_metrics[i,1], i*SAMPLING_RATE, 'abs', True)
        plot_stock(test_y.index, test_y, test_pred[i], d, test_metrics[i,0], test_metrics[i,1], i*SAMPLING_RATE, 'prediction', False)
        plot_stock(test_y.index, test_y, test_pred[i], d, test_metrics[i,0], test_metrics[i,1], i*SAMPLING_RATE, 'abs', False)
    train_metrics_df = pd.DataFrame(train_metrics, index=np.arange(0,n_epochs+1,SAMPLING_RATE),columns=['RMSE','MAE','HITRATE'])
    train_metrics_df.to_csv(d+'train/metrics.csv')
    test_metrics_df = pd.DataFrame(test_metrics, index=np.arange(0,n_epochs+1,SAMPLING_RATE),columns=['RMSE','MAE','HITRATE'])
    test_metrics_df.to_csv(d+'test/metrics.csv')
    
    for i in range(n_ensemble):
        saver = tf.train.Saver()
        with tf.Session() as sess:
            saver.restore(sess, "{0}model_{1}/model.ckpt".format(d, i))
            sta = time.time()
            prediction = make_prediction(test_x, test_y, sess, BATCH_SIZE)
            run = time.time() - sta
            pred_time = pred_time + (run - pred_time)/(i+1)
            r, m, h = cal_metrics(test_y.as_matrix(), prediction)
            plot_stock(test_y.index, test_y, prediction, d, r, m, best_iteration[i,0], 'prediction', False)
            plot_stock(test_y.index, y_test, prediction, d, r, m, best_iteration[i,0], 'abs', False)
            best_pred = best_pred + (prediction - best_pred)/(i+1)
    rmse, mae, hit = cal_metrics(test_y.as_matrix(), best_pred)
    print ("Prediction time = {}".format(pred_time))
    print (" *** RMSE={:7.5f}, MAE={:7.5f}, HIT={:7.5f} ***".format(rmse, mae, hit))

        

In [13]:
df_train_d = helper.inter_polation_norm_1_1(df_train)
n_ensembles = 10

In [14]:
# learning_rate = tf.placeholder(tf.float32, shape=[])
# def decay_learning_rate(learning_step, total_step):
#     ini = 1e-3
#     if(learning_step <= 3000):
#         return ini
#     return ini*(1-(learning_step-3000)/total_step)

# cost = tf.reduce_mean(tf.square(hidden_layer_1-Y))
# optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
# y_pred = hidden_layer_1
# y_true = Y
# ## Trigonometric expansion
# df_train_data = helper.trignometric_expansion(df_train_d, 4)
# df_target = df.loc[df_train_data.index][['Close']]
# df_target = helper.inter_polation_norm_1_1(df_target)
# d = './FLANN/tri_decay/'
# ## Train test split
# x_train, y_train = df_train_data.loc[:'2014-01-01'].as_matrix(), df_target.loc[:'2014-01-01']
# x_test, y_test  = df_train_data.loc['2014-01-01':].as_matrix(), df_target.loc['2014-01-01':]
# train_ensemble(10, x_train, y_train, x_test, y_test , TRAINING_EPOCHS, BATCH_SIZE, d)


In [15]:
cost = tf.reduce_mean(tf.square(hidden_layer_1-Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
y_pred = hidden_layer_1
y_true = Y
df_train_d = helper.inter_polation_norm_1_1(df_train)
df_train_data = helper.trignometric_expansion(df_train_d, 4)
df_target = df.loc[df_train_data.index][['Close']]
df_target = helper.inter_polation_norm_1_1(df_target)
d = './FLANN/tri_decay/'
## Train test split
x_train, y_train = df_train_data.loc[:'2014-01-01'].as_matrix(), df_target.loc[:'2014-01-01']
x_test, y_test  = df_train_data.loc['2014-01-01':].as_matrix(), df_target.loc['2014-01-01':]
train_ensemble(n_ensembles, x_train, y_train, x_test, y_test , TRAINING_EPOCHS, BATCH_SIZE, d)


df_train_data = helper.chebyshev_expansion(df_train_d, 8)
df_target = df.loc[df_train_data.index][['Close']]
df_target = helper.inter_polation_norm_1_1(df_target)
## Train test split
d = './FLANN/che_decay/'
x_train, y_train = df_train_data.loc[:'2014-01-01'].as_matrix(), df_target.loc[:'2014-01-01']
x_test, y_test  = df_train_data.loc['2014-01-01':].as_matrix(), df_target.loc['2014-01-01':]
train_ensemble(n_ensembles, x_train, y_train, x_test, y_test , TRAINING_EPOCHS, BATCH_SIZE, d)

df_train_data = helper.legendre_expansion(df_train_d, 8)
df_target = df.loc[df_train_data.index][['Close']]
df_target = helper.inter_polation_norm_1_1(df_target)
## Train test split
d = './FLANN/leg_decay/'
x_train, y_train = df_train_data.loc[:'2014-01-01'].as_matrix(), df_target.loc[:'2014-01-01']
x_test, y_test  = df_train_data.loc['2014-01-01':].as_matrix(), df_target.loc['2014-01-01':]
train_ensemble(n_ensembles, x_train, y_train, x_test, y_test , TRAINING_EPOCHS, BATCH_SIZE, d)

Esemble 0
Training.......
---Total Running time: 511.54350757598877 seconds ---
**************************************************
Esemble 1
Training.......
---Total Running time: 519.9518010616302 seconds ---
**************************************************
Esemble 2
Training.......
---Total Running time: 517.5722782611847 seconds ---
**************************************************
Esemble 3
Training.......
---Total Running time: 514.9747905731201 seconds ---
**************************************************
Esemble 4
Training.......
---Total Running time: 468.4525315761566 seconds ---
**************************************************
Esemble 5
Training.......
---Total Running time: 503.239946603775 seconds ---
**************************************************
Esemble 6
Training.......
---Total Running time: 503.8340001106262 seconds ---
**************************************************
Esemble 7
Training.......
---Total Running time: 579.4184858798981 seconds ---
***********

In [16]:
beta = 0.0001
learning_rate = tf.placeholder(tf.float32, shape=[])
def decay_learning_rate(learning_step, total_step):
    ini = 1e-3
    if(learning_step <= 3000):
        return ini
    return ini*(1-(learning_step-3000)/total_step)

cost = tf.reduce_mean(tf.square(hidden_layer_1-Y)) + beta*tf.nn.l2_loss(weight_1)
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
y_pred = hidden_layer_1
y_true = Y
## Trigonometric expansion
df_train_data = helper.trignometric_expansion(df_train_d, 4)
df_target = df.loc[df_train_data.index][['Close']]
df_target = helper.inter_polation_norm_1_1(df_target)
d = './FLANN/tri_decay_L2/'
## Train test split
x_train, y_train = df_train_data.loc[:'2014-01-01'].as_matrix(), df_target.loc[:'2014-01-01']
x_test, y_test  = df_train_data.loc['2014-01-01':].as_matrix(), df_target.loc['2014-01-01':]
train_ensemble(n_ensembles, x_train, y_train, x_test, y_test , TRAINING_EPOCHS, BATCH_SIZE, d)

## Chebyshev expansion
df_train_data = helper.chebyshev_expansion(df_train_d, 8)
df_target = df.loc[df_train_data.index][['Close']]
df_target = helper.inter_polation_norm_1_1(df_target)
## Train test split
d = './FLANN/che_decay_L2/'
x_train, y_train = df_train_data.loc[:'2014-01-01'].as_matrix(), df_target.loc[:'2014-01-01']
x_test, y_test  = df_train_data.loc['2014-01-01':].as_matrix(), df_target.loc['2014-01-01':]
train_ensemble(n_ensembles, x_train, y_train, x_test, y_test , TRAINING_EPOCHS, BATCH_SIZE, d)

## Legendre expansion
df_train_data = helper.legendre_expansion(df_train_d, 8)
df_target = df.loc[df_train_data.index][['Close']]
df_target = helper.inter_polation_norm_1_1(df_target)
## Train test split
d = './FLANN/leg_decay_L2/'
x_train, y_train = df_train_data.loc[:'2014-01-01'].as_matrix(), df_target.loc[:'2014-01-01']
x_test, y_test  = df_train_data.loc['2014-01-01':].as_matrix(), df_target.loc['2014-01-01':]
train_ensemble(n_ensembles, x_train, y_train, x_test, y_test , TRAINING_EPOCHS, BATCH_SIZE, d)

Esemble 0
Training.......
---Total Running time: 985.2641668319702 seconds ---
**************************************************
Esemble 1
Training.......
---Total Running time: 902.734424829483 seconds ---
**************************************************
Esemble 2
Training.......
---Total Running time: 872.7211511135101 seconds ---
**************************************************
Esemble 3
Training.......
---Total Running time: 926.3221204280853 seconds ---
**************************************************
Esemble 4
Training.......
---Total Running time: 945.5492050647736 seconds ---
**************************************************
Esemble 5
Training.......
---Total Running time: 801.8306787014008 seconds ---
**************************************************
Esemble 6
Training.......
---Total Running time: 893.1008937358856 seconds ---
**************************************************
Esemble 7
Training.......
---Total Running time: 1020.1596300601959 seconds ---
***********

In [17]:
TRAINING_EPOCHS = 20000
SAMPLING_RATE = 500
## Cost function and optimization
# BETA = 0.001 #L2 regularization penalty factor
LEARNING_RATE = 1e-3
cost = tf.reduce_mean(tf.square(hidden_layer_1-Y)) # To be added L2 regularization
optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE).minimize(cost)
## Making Prediction
y_pred = hidden_layer_1
y_true = Y
df_train_d = helper.inter_polation_norm_1_1(df_train)
df_train_data = helper.trignometric_expansion(df_train_d, 4)
df_target = df.loc[df_train_data.index][['Close']]
df_target = helper.inter_polation_norm_1_1(df_target)
d = './FLANN/tri_fix/'
## Train test split
x_train, y_train = df_train_data.loc[:'2014-01-01'].as_matrix(), df_target.loc[:'2014-01-01']
x_test, y_test  = df_train_data.loc['2014-01-01':].as_matrix(), df_target.loc['2014-01-01':]
train_ensemble(n_ensembles, x_train, y_train, x_test, y_test , TRAINING_EPOCHS, BATCH_SIZE, d)


df_train_data = helper.chebyshev_expansion(df_train_d, 8)
df_target = df.loc[df_train_data.index][['Close']]
df_target = helper.inter_polation_norm_1_1(df_target)
## Train test split
d = './FLANN/che_fix/'
x_train, y_train = df_train_data.loc[:'2014-01-01'].as_matrix(), df_target.loc[:'2014-01-01']
x_test, y_test  = df_train_data.loc['2014-01-01':].as_matrix(), df_target.loc['2014-01-01':]
train_ensemble(n_ensembles, x_train, y_train, x_test, y_test , TRAINING_EPOCHS, BATCH_SIZE, d)

df_train_data = helper.legendre_expansion(df_train_d, 8)
df_target = df.loc[df_train_data.index][['Close']]
df_target = helper.inter_polation_norm_1_1(df_target)
## Train test split
d = './FLANN/leg_fix/'
x_train, y_train = df_train_data.loc[:'2014-01-01'].as_matrix(), df_target.loc[:'2014-01-01']
x_test, y_test  = df_train_data.loc['2014-01-01':].as_matrix(), df_target.loc['2014-01-01':]
train_ensemble(n_ensembles, x_train, y_train, x_test, y_test , TRAINING_EPOCHS, BATCH_SIZE, d)

Esemble 0
Training.......
---Total Running time: 1413.8336727619171 seconds ---
**************************************************
Esemble 1
Training.......
---Total Running time: 2287.465895175934 seconds ---
**************************************************
Esemble 2
Training.......
---Total Running time: 1782.713847875595 seconds ---
**************************************************
Esemble 3
Training.......
---Total Running time: 1841.846482515335 seconds ---
**************************************************
Esemble 4
Training.......
---Total Running time: 1624.6334261894226 seconds ---
**************************************************
Esemble 5
Training.......
---Total Running time: 2330.9864428043365 seconds ---
**************************************************
Esemble 6
Training.......
---Total Running time: 1901.4000971317291 seconds ---
**************************************************
Esemble 7
Training.......
---Total Running time: 2691.6738991737366 seconds ---
******